In [36]:
import pandas as pd
import random
from collections import defaultdict

# 读取数据
data = pd.read_csv('结果4.csv')

# 定义班次时间范围
shifts = {
    "00:00-08:00": range(0, 8),
    "05:00-13:00": range(5, 13),
    "08:00-16:00": range(8, 16),
    "12:00-20:00": range(12, 20),
    "14:00-22:00": range(14, 22),
    "16:00-24:00": range(16, 24)
}

# 遗传算法参数
num_generations = 30
population_size = 50
num_workers = 60
num_shifts = 6

# 循环每个分拣中心执行优化
for center in data['分拣中心'].unique():
    # 过滤特定分拣中心的数据
    center_data = data[data['分拣中心'] == center]

    # 按日期和班次整理每小时数据
    shift_data = {}
    for shift, hours in shifts.items():
        mask = center_data['小时'].isin(hours)
        grouped = center_data.loc[mask].groupby('日期')['货量'].sum()
        shift_data[shift] = grouped.to_dict()

    dates = sorted(center_data['日期'].unique())

    # 染色体表示和适应度函数
    def create_chromosome():
        chromosome = {}
        for date in dates:
            workers_daily_schedule = {worker: None for worker in range(num_workers)}
            for shift_key in shifts.keys():
                available_workers = [w for w, s in workers_daily_schedule.items() if s is None]
                num_regulars = random.randint(1, min(10, len(available_workers)))
                regulars = random.sample(available_workers, k=num_regulars)
                for reg in regulars:
                    workers_daily_schedule[reg] = shift_key
                
                chromosome[(date, shift_key)] = {
                    'regulars': regulars,
                    'temps': random.randint(1, 5)
                }
        return chromosome

    def calculate_fitness(chromosome):
        fitness = 0
        total_worker_days = 0  # 新增变量，用于计算总人天数
        worker_days = defaultdict(int)
    
        for (day, shift_key), info in chromosome.items():
            for worker in info['regulars']:
                worker_days[worker] += 1
    
            # 计算当前班次的总出勤人数（正式工 + 临时工）
            total_workers = len(info['regulars']) + info['temps']
            total_worker_days += total_workers  # 累加总人天数
    
            # 原有的适应度评估
            fitness += len(info['regulars']) * 1 + info['temps'] * 0.8
            required_output = shift_data[shift_key].get(day, 0)
            cargo_met = 25 * len(info['regulars']) + 20 * info['temps']
            if cargo_met < required_output:
                deficit = required_output - cargo_met
                fitness -= deficit * 10
            else:
                fitness += required_output
    
        # 将总人天数的影响合并到适应度计算中，由于目标是最小化总人天数，
        # 我们可以将总人天数作为一个惩罚项添加到适应度中
        fitness += total_worker_days  # 可以调整这里的权重以强调总人天数的影响
    
        return fitness


 

In [37]:
# 选择
def select(population):
    sorted_population = sorted(population, key=calculate_fitness)
    return sorted_population[:len(population)//2]


# 交叉
def crossover(chromosome1, chromosome2):
    child = chromosome1.copy()
    crossover_point = random.randint(0, len(chromosome1))
    crossover_keys = list(chromosome1.keys())[crossover_point:]
    for key in crossover_keys:
        child[key] = chromosome2[key]
    return child

# 变异
def mutate(chromosome):
    mutation_key = random.choice(list(chromosome.keys()))
    chromosome[mutation_key]['regulars'] = random.sample(range(num_workers), k=random.randint(1, 10))
    chromosome[mutation_key]['temps'] = random.randint(1, 5)
    return chromosome


In [38]:
def genetic_algorithm():
    population = [create_chromosome() for _ in range(population_size)]
    for generation in range(num_generations):
        population = select(population)
        new_generation = []
        while len(new_generation) < population_size:
            parent1, parent2 = random.sample(population, 2)
            child = crossover(parent1, parent2)
            if random.random() < 0.04:
                child = mutate(child)
            new_generation.append(child)
        population = new_generation
    # 返回适应度最低的染色体，因为我们想最小化总人天数
    return min(population, key=calculate_fitness)
genetic_algorithm()


{('2023/12/01', '00:00-08:00'): {'regulars': [4], 'temps': 4},
 ('2023/12/01', '05:00-13:00'): {'regulars': [57, 51], 'temps': 2},
 ('2023/12/01', '08:00-16:00'): {'regulars': [35], 'temps': 5},
 ('2023/12/01', '12:00-20:00'): {'regulars': [27], 'temps': 4},
 ('2023/12/01', '14:00-22:00'): {'regulars': [39], 'temps': 3},
 ('2023/12/01', '16:00-24:00'): {'regulars': [9, 42, 32, 0, 49, 48, 2, 37],
  'temps': 3},
 ('2023/12/02', '00:00-08:00'): {'regulars': [39, 31, 46, 55], 'temps': 4},
 ('2023/12/02', '05:00-13:00'): {'regulars': [17, 32], 'temps': 1},
 ('2023/12/02', '08:00-16:00'): {'regulars': [8, 22, 25], 'temps': 5},
 ('2023/12/02', '12:00-20:00'): {'regulars': [41, 56, 38, 44, 7, 2, 40],
  'temps': 2},
 ('2023/12/02', '14:00-22:00'): {'regulars': [52, 47, 9], 'temps': 1},
 ('2023/12/02', '16:00-24:00'): {'regulars': [50, 33, 15, 4, 40], 'temps': 4},
 ('2023/12/03', '00:00-08:00'): {'regulars': [8, 38, 59, 29, 16, 48, 5, 56],
  'temps': 3},
 ('2023/12/03', '05:00-13:00'): {'regular

In [40]:
import pandas as pd

def genetic_algorithm_for_all_centers():
    results = {}
    for center in data['分拣中心'].unique():
        # 过滤特定分拣中心的数据
        center_data = data[data['分拣中心'] == center]

        # 按日期和班次整理每小时数据
        shift_data = {}
        for shift, hours in shifts.items():
            mask = center_data['小时'].isin(hours)
            grouped = center_data.loc[mask].groupby('日期')['货量'].sum()
            shift_data[shift] = grouped.to_dict()

        dates = sorted(center_data['日期'].unique())

        population = [create_chromosome() for _ in range(population_size)]
        for generation in range(num_generations):
            population = select(population)
            new_generation = []
            while len(new_generation) < population_size:
                parent1, parent2 = random.sample(population, 2)
                child = crossover(parent1, parent2)
                if random.random() < 0.04:
                    child = mutate(child)
                new_generation.append(child)
            population = new_generation
        best_solution = min(population, key=calculate_fitness)
        results[center] = best_solution
    return results

best_chromosomes = genetic_algorithm_for_all_centers()

columns = ['分拣中心', '日期', '班次', '正式工人数', '临时工人数']
results_df = pd.DataFrame(columns=columns)
shift_counts = {}

for center, best_chromosome in best_chromosomes.items():
    for (date, shift), value in best_chromosome.items():
        key = (center, date, shift)
        if key not in shift_counts:
            shift_counts[key] = {'正式工人数': 0, '临时工人数': 0}
        shift_counts[key]['正式工人数'] += len(value['regulars'])
        shift_counts[key]['临时工人数'] += value['temps']

rows = [
    {'分拣中心': k[0], '日期': k[1], '班次': k[2], '正式工人数': v['正式工人数'], '临时工人数': v['临时工人数']}
    for k, v in shift_counts.items()
]
results_df = pd.DataFrame(rows, columns=columns)

results_df.head(20)


,分拣中心,日期,班次,正式工人数,临时工人数
0,SC54,2023/12/01,00:00-08:00,7,1
1,SC54,2023/12/01,05:00-13:00,8,2
2,SC54,2023/12/01,08:00-16:00,2,3
3,SC54,2023/12/01,12:00-20:00,3,2
4,SC54,2023/12/01,14:00-22:00,2,4
5,SC54,2023/12/01,16:00-24:00,3,2
6,SC54,2023/12/02,00:00-08:00,8,5
7,SC54,2023/12/02,05:00-13:00,2,1
8,SC54,2023/12/02,08:00-16:00,1,3
9,SC54,2023/12/02,12:00-20:00,4,4


In [41]:
results_df.to_csv('结果5.csv', index=False, encoding='utf-8-sig')

print("数据已经成功保存到结果5.csv文件中。")

数据已经成功保存到结果5.csv文件中。
